In [ ]:
import requests
from bs4 import BeautifulSoup
import os

In [ ]:
def get_data():
    link = input('Введите ссылку с главной странички на jut.su нужного вам аниме - ')
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36 OPR/84.0.4316.42 (Edition avira-2)'}
    req = requests.get(link, headers=headers).text
    soup = BeautifulSoup(req, 'lxml')
    return soup

In [ ]:
def get_links(soup):

    episodes = []
    black = soup.find_all(class_='short-btn black video the_hildi')
    green = soup.find_all(class_='short-btn green video the_hildi')
    for i in black:
        episodes.append(str(i))
    for i in green:
        episodes.append(str(i))

    links = []
    for i in episodes:
        start = 0
        finish = 0
        for h in range(5, len(i)):
            if i[h-4:h] == 'href':
                start = h+3
            if i[h-4:h] == 'html':
                finish = h
        link = 'https://jut.su/' + i[start:finish]
        links.append(link)

    season = soup.find_all(class_='b-b-title the-anime-season center')
    name = soup.find_all('title')
    if len(season) > 0:
        title = str(name[0])[16:-37]
    else:
        title = str(name[0])[16:-28]

    return title, links

In [ ]:
def create(links, title):
    content = []
    for i in links:
        video = {
            'title': title,
            'season': 0,
            'episode': 0,
            'link': '',
            'res': 1080
        }
        for h in range(5, len(i)):
            if i[h-4:h] == 'film':
                video['season'] = 0
                video['episode'] = int(i[h+1])
            if i[h-7:h] == 'episode' and i[h-9] == '-':
                video['season'] = int(i[h-9])
                x = ''
                while x[-1] != '.':
                    x += i[h] + i[h+len(x)]
                video['episode'] = int(x[1:-1])
            elif i[h-7:h] == 'episode' and i[h-9] != '-':
                video['season'] = 1
                x = i[h]
                while x[-1] != '.':
                    x += i[h+len(x)]
                video['episode'] = int(x[1:-1])
            video['link'] = i
        content.append(video)
    return content

In [ ]:
def ask_user(data, loadlist):
    categories = []
    for i in data:
        if i['season'] not in categories:
            categories.append(i['season'])
    if 0 in categories:
        file = input(f'''Какой сезон вы хотите скачать?
        Для скачивания доступно {len(categories)-1} сезонов и
        полнометражные фильмы (Сезон 0)
        Введите номер нужного сезона
        Чтобы скачать все сезоны сразу, введите 'all' (не рекомендуется)''')
    else:
        file = input(f'''Какой сезон вы хотите скачать?
        Для скачивания доступно {len(categories)} сезонов
        Введите номер нужного сезона
        Чтобы скачать все сезоны сразу, введите 'all' (не рекомендуется)''')

    if file == 'all':
        for i in data:
            if i not in loadlist:
                loadlist.append(i)
    else:
        episodes = []
        for i in data:
            if i['season'] == int(file):
                episodes.append(i)
        file = input(f'''Какую серию вы хотите скачать?
        Для скачивания доступно {len(episodes)} серий
        Введите номер нужной серии
        Чтобы скачать все серии сразу, введите 'all' (не рекомендуется)''')
        if file == 'all':
            for i in episodes:
                if i not in loadlist:
                    loadlist.append(i)
        else:
            for i in episodes:
                if i['episode'] == int(file) and i not in loadlist:
                    loadlist.append(i)
    re = input('''Хотите скачать что-либо ещё? (+/-)''')
    if re == '+':
        main(loadlist)
    elif re == '-':
        return loadlist
    else:
        print('Error')

In [ ]:
def load(loadlist):
    for i in loadlist:
        headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36 OPR/84.0.4316.42 (Edition avira-2)'}
        req = requests.get(i['link'], headers=headers).text
        soup = BeautifulSoup(req, 'lxml')
        direct = str(soup.find('source'))[48:-20]

        if not os.path.exists(f'./{i["title"]}'):
            os.mkdir(f'./{i["title"]}')
        if not os.path.exists(f'./{i["title"]}/Сезон {i["season"]}'):
            os.mkdir(f'./{i["title"]}/Сезон {i["season"]}')

        if not os.path.exists(f'{i["title"]}/Сезон {i["season"]}/Серия {i["episode"]}.mp4'):
            r = requests.get(direct, headers=headers)
            open(f'{i["title"]}/Сезон {i["season"]}/Серия {i["episode"]}.mp4', 'wb').write(r.content)
            print(f'Файл "{i["title"]}, Сезон {i["season"]}, Серия {i["episode"]}" успешно загружен. Осталось загрузить {len(loadlist) - loadlist.index(i) - 1} файлов из {len(loadlist)}')
        else:
            print(f'Файл "{i["title"]}, Сезон {i["season"]}, Серия {i["episode"]}" и так был загружен. Осталось загрузить {len(loadlist) - loadlist.index(i) - 1} файлов из {len(loadlist)}')

In [ ]:
def main(loadlist):
    soup = get_data()
    title, links = get_links(soup)
    data = create(links, title)
    ask_user(data, loadlist)
    load(loadlist)


if __name__ == '__main__':
    loadlist = []
    main(loadlist)

Файл "Такт Опус. Судьба, Сезон 1, Серия 2" успешно загружен. Осталось загрузить 10 файлов из 12
https://r310104.kujo-jotaro.com/takt-op-destiny/3.1080.d9b4bdfe805afb86.mp4?hash1=ca37d94e988f307c5dafb7dc965f18eb&amp;hash2=508f620ba560c9adc0cb3dfe482c1fef
Файл "Такт Опус. Судьба, Сезон 1, Серия 3" успешно загружен. Осталось загрузить 9 файлов из 12
https://r63.kujo-jotaro.com/takt-op-destiny/4.1080.9acd0c000b96aea7.mp4?hash1=3196838d58d47c657c7d98455f5d8b89&amp;hash2=dfaa9c1c8d04b044eee9d001efe1b112
